In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 100

In [2]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Избавляемся от признака Healthcare_1, так как половина значений в нем пропущены

In [3]:
data = data.drop(['Healthcare_1'], axis=1)
test = test.drop(['Healthcare_1'], axis=1)

C DistrictID долго мучался. Решил не использовать.

In [4]:
#data.DistrictId=data.DistrictId.astype(str)
data = data.drop(['DistrictId'], axis=1)
test = test.drop(['DistrictId'], axis=1)

Избавляемся от выбросов

In [5]:
data.loc[data.HouseYear == 20052011, 'HouseYear'] = 2008
data.loc[data.HouseYear == 4968, 'HouseYear'] = 1968

In [6]:
data = data[data['Square'] < 300]

In [7]:
data.loc[data['Rooms'] == 0, 'Rooms'] = None
test.loc[test['Rooms'] == 0, 'Rooms'] = None

In [8]:
data.loc[data['Square'] < 15, 'Square'] = None
test.loc[test['Square'] < 15, 'Square'] = None

In [9]:
data.loc[data['LifeSquare'] < 15, 'LifeSquare'] = None
test.loc[test['LifeSquare'] < 15, 'LifeSquare'] = None

In [10]:
data.loc[data['LifeSquare'] > 250, 'LifeSquare'] = None
test.loc[test['LifeSquare'] > 250, 'LifeSquare'] = None

In [11]:
data.loc[data['Rooms'] > 5, 'Rooms'] = None
test.loc[test['Rooms'] > 5, 'Rooms'] = None

In [12]:
data.loc[data['KitchenSquare'] > 200, 'KitchenSquare'] = None
test.loc[test['KitchenSquare'] > 200, 'KitchenSquare'] = None

In [13]:
data = pd.get_dummies(data)
test = pd.get_dummies(test)

Вставляем пропущенные значения с помощью sklearn.impute

In [14]:
imputer = IterativeImputer(random_state=100, n_nearest_features=5)
imp_train_data = pd.concat([data, test]).drop(['Id', 'Price'], axis=1)
full_completed_data = pd.DataFrame(imputer.fit_transform(imp_train_data), columns=imp_train_data.columns)

In [15]:
data = data.fillna(full_completed_data)

In [16]:
test = test.fillna(full_completed_data.tail(5000).reset_index(drop=True))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['Id', 'Price'], axis=1),
    data['Price'],
    test_size=0.3,
    random_state=100
)

In [18]:
parameters = {
    'n_estimators': [100],
    'max_depth': np.arange(10, 20),
    'max_features': np.arange(5, 15),
    'min_samples_leaf': np.arange(2, 10)
}

clf = GridSearchCV(
    estimator=RandomForestRegressor(random_state=100),
    param_grid=parameters,
    scoring='r2',
    n_jobs = -1,
    cv=5
)

In [19]:
np.arange(4, 20, 3)

array([ 4,  7, 10, 13, 16, 19])

In [20]:
%%time
clf.fit(X_train, y_train)

Wall time: 10min 59s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=100,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': array([10, 11, 12, 13

In [21]:
clf.best_params_

{'max_depth': 19,
 'max_features': 5,
 'min_samples_leaf': 2,
 'n_estimators': 100}

In [22]:
best_clf = RandomForestRegressor(**clf.best_params_, random_state=100)
best_clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
                      max_features=5, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=100, verbose=0,
                      warm_start=False)

In [23]:
y_pred = best_clf.predict(X_test)

In [24]:
from sklearn.metrics import r2_score as r2
r2(y_test, y_pred)

0.7277589103207802

Предсказываем основные результаты

In [25]:
model = RandomForestRegressor(**clf.best_params_, random_state=100)

In [26]:
model.fit(data.drop(['Id', 'Price'], axis=1), data['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
                      max_features=5, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=100, verbose=0,
                      warm_start=False)

In [27]:
pred_prices = model.predict(test.drop('Id', axis=1))

In [28]:
test['Price'] = pred_prices

In [29]:
test[['Id','Price']].to_csv('DBatorov_predictions.csv', index=None)